# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import joblib, pickle
import json

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

#from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.33.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-project'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster = 'automl-cluster'

try:
    comp_trget = ComputeTarget(workspace=ws, name=compute_cluster)
    print('Cluster already exists')
except ComputeTargetException:
    compute_confg = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",max_nodes=4) ## VER
    comp_trget = ComputeTarget.create(ws,compute_cluster,compute_confg)

In [21]:
data_link = 'https://raw.githubusercontent.com/fcgomezr/Udacity-Project-3/main/data%20sets/WA_Fn-UseC_-Telco-Customer-Churn.csv'
input_data = TabularDatasetFactory.from_delimited_files(path=data_link, separator = ",")

In [22]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(input_data)
predict_col = pd.DataFrame(y)
predict_col.head()
complete_data = pd.concat([x,predict_col],axis=1)
complete_data.head()

,customerID,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,gender_Male,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Contract_One year,Contract_Two year,Churn
0,7590-VHVEG,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,5575-GNVDE,0,0,0,34,1,0,1,0,1,...,1,0,0,0,0,0,1,1,0,0
2,3668-QPYBK,0,0,0,2,1,0,1,1,0,...,1,0,0,0,0,0,1,0,0,1
3,7795-CFOCW,0,0,0,45,0,0,1,0,1,...,1,0,0,1,0,0,0,1,0,0
4,9237-HQITU,0,0,0,2,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1


In [24]:
# split data into training and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(complete_data,test_size=0.25)
train_data.head()

,customerID,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,gender_Male,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Contract_One year,Contract_Two year,Churn
3989,7634-WSWDB,0,0,1,8,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1220,0872-JCPIB,0,0,0,17,1,1,1,0,0,...,1,0,0,1,0,0,0,0,0,0
6362,6856-RAURS,0,0,0,2,1,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
6081,8757-TFHHJ,0,0,0,1,1,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
5623,9115-YQHGA,0,0,0,40,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0


In [25]:
train_data.to_csv("traindata.csv")

In [26]:
data_store = ws.get_default_datastore()
data_store.upload(src_dir='./',target_path='./')
ftrain_data = Dataset.Tabular.from_delimited_files(path = [(data_store, './traindata.csv')])

Uploading an estimated of 19 files
Target already exists. Skipping upload for .amlignore
Target already exists. Skipping upload for .amlignore.amltmp
Target already exists. Skipping upload for automl.ipynb
Target already exists. Skipping upload for hyperparameter_tuning (1).ipynb
Target already exists. Skipping upload for hyperparameter_tuning.ipynb
Target already exists. Skipping upload for model_training.py
Target already exists. Skipping upload for README.md
Target already exists. Skipping upload for score.py
Target already exists. Skipping upload for train.py
Target already exists. Skipping upload for traindata.csv
Target already exists. Skipping upload for .ipynb_checkpoints/automl-checkpoint.ipynb
Uploading ./automl.log
Uploaded ./automl.log, 1 files out of an estimated total of 16
Uploading ./automl_errors.log
Uploaded ./automl_errors.log, 2 files out of an estimated total of 16
Uploading ./azureml_automl.log
Uploaded ./azureml_automl.log, 3 files out of an estimated total of 16

In [30]:
temp = ftrain_data.drop_columns('Column1')

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [34]:
# TODO: Put your automl settings here
automl_settings = {    
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             compute_target=comp_trget,
                             task = "classification",
                             training_data=temp,
                             label_column_name="Churn",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             n_cross_validations=7,
                             debug_log = "automl_errors.log",
                             **automl_settings
)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)


Submitting remote run.
No run_configuration provided, running on automl-cluster with default configuration
Running on remote compute: automl-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project,AutoML_22f48fe3-4094-4615-b682-695c86b1a332,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project,AutoML_aa0f197a-bf52-486a-ac58-1b41f571364e,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization
DETAILS:      
+---------------------------------+---------------------------------+--------------------------------

{'runId': 'AutoML_aa0f197a-bf52-486a-ac58-1b41f571364e',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-09-24T04:01:50.806418Z',
 'endTimeUtc': '2021-09-24T04:31:31.873562Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '7',
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-project","subscription_id":"a0a76bad-11a1-4a2d-9887-97a29122c8ed","resource_group":"aml-quickstarts-159151","workspace_name":"quick-starts-ws-159151","region":"southcentralus","compute_target":"automl-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metri

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_automl, best_fit_model = remote_run.get_output()
print(best_fit_model)

Package:azureml-automl-runtime, training version:1.34.0, current version:1.33.0
Package:azureml-core, training version:1.34.0, current version:1.33.0
Package:azureml-dataprep, training version:2.22.2, current version:2.20.1
Package:azureml-dataprep-rslex, training version:1.20.2, current version:1.18.0
Package:azureml-dataset-runtime, training version:1.34.0, current version:1.33.0
Package:azureml-defaults, training version:1.34.0, current version:1.33.0
Package:azureml-interpret, training version:1.34.0, current version:1.33.0
Package:azureml-mlflow, training version:1.34.0, current version:1.33.0
Package:azureml-pipeline-core, training version:1.34.0, current version:1.33.0
Package:azureml-responsibleai, training version:1.34.0, current version:1.33.0
Package:azureml-telemetry, training version:1.34.0, current version:1.33.0
Package:azureml-train-automl-client, training version:1.34.0, current version:1.33.0
Package:azureml-train-automl-runtime, training version:1.34.0, current versi

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=0.4166666666666667, reg_lambda=0, subsample=0.5, tree_method='auto'))], verbose=False))], flatten_transform=None, weights=[0.15384615384615385, 0.23076923076923078, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693]))],
         verbose=False)
Y_transformer(['LabelEncoder', LabelEncoder()])


In [14]:
print(best_automl)

Run(Experiment: capstone-project,
Id: AutoML_aa0f197a-bf52-486a-ac58-1b41f571364e_44,
Type: azureml.scriptrun,
Status: Completed)


In [15]:
#TODO: Save the best model
joblib.dump(best_fit_model, 'best_fit_automl_model.pkl')

['best_fit_automl_model.pkl']

In [18]:
#joblib.dump(best_fit_model,'./outputs/bestmodel.joblib')
#best_class.register_model(model_name='best_fit_automl_model.pkl',model_path='/outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
from azureml.core.model import Model
model = Model.register(workspace = ws, model_name = 'best_fit_automl_model', model_path = 'best_fit_automl_model.pkl')
print(model.name, model.id, model.version, sep='\t')

Registering model best_fit_automl_model
best_fit_automl_model	best_fit_automl_model:1	1


TODO: In the cell below, send a request to the web service you deployed to test it.

In [17]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, Webservice, AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import azureml.train.automl

# Create the environment
env = best_automl.get_environment()
conda_dep = CondaDependencies()

# Define the packages needed by the model and the script
#conda_dep.add_conda_package("numpy")
#conda_dep.add_conda_package("scikit-learn")
#conda_dep.add_conda_package("pandas")
# Define azureml-defaults as a pip dependency
#conda_dep.add_pip_package("azureml-defaults")
#conda_dep.add_pip_package("azureml-train")

# Adds dependencies to PythonSection of myenv
#env.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script='score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)
service = Model.deploy(ws, "customerservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-24 04:46:41+00:00 Creating Container Registry if not exists..
2021-09-24 04:56:42+00:00 Registering the environment.
2021-09-24 04:56:42+00:00 Use the existing image.
2021-09-24 04:56:42+00:00 Generating deployment configuration..
2021-09-24 04:56:49+00:00 Submitting deployment to compute..
2021-09-24 04:56:51+00:00 Checking the status of deployment customerservice..
2021-09-24 05:02:26+00:00 Checking the status of inference endpoint customerservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://5ab250f3-7fd4-4383-8c40-22990a95eb8e.southcentralus.azurecontainer.io/score
http://5ab250f3-7fd4-4383-8c40-22990a95eb8e.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, print the logs of the web service and delete the service

In [18]:
import requests
import json

# URL for the web service
scoring_uri = 'http://5ab250f3-7fd4-4383-8c40-22990a95eb8e.southcentralus.azurecontainer.io/score'

# Set the content type
headers = {'Content-Type': 'application/json'}

# Two sets of data to score, so we get two results back
data = {"data":
        [{
         
            "gender":"Female",
            "SeniorCitizen":0,
            "Partner":"Yes",
            "Dependents":"No",
            "tenure":1,
            "PhoneService":"No",
            "MultipleLines":"No phone service",
            "InternetService":"DSL",
            "OnlineSecurity":"No",
            "OnlineBackup":"Yes",
            "DeviceProtection":"No",
            "TechSupport":"No",
            "StreamingTV":"No",
            "StreamingMovies":"No",
            "Contract":"Month-to-month",
            "PaperlessBilling":"Yes",
            "PaymentMethod":"Electronic check",
            "MonthlyCharges":29.85,
            "TotalCharges":29.85}
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)

print("Response Code : ", resp.status_code)
print("Predicted Value : ",resp.text)

Response Code :  200
Predicted Value :  "DataException:\n\tMessage: The number of features in [fitted data](21) does not match with those in [input data](19). Please inspect your data, and make sure that features are aligned in both the Datasets.\n\tInnerException: None\n\tErrorResponse \n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"The number of features in [fitted data](21) does not match with those in [input data](19). Please inspect your data, and make sure that features are aligned in both the Datasets.\",\n        \"target\": \"X\",\n        \"inner_error\": {\n            \"code\": \"BadData\",\n            \"inner_error\": {\n                \"code\": \"InvalidDimension\",\n                \"inner_error\": {\n                    \"code\": \"DataShapeMismatch\"\n                }\n            }\n        },\n        \"reference_code\": \"c402b6c2-3870-45a7-8745-c063bd385962\"\n    }\n}"


In [25]:
# Web Service Logs
print(service.get_logs())

2021-09-18T18:53:07,813563500+00:00 - iot-server/run 
2021-09-18T18:53:07,807115300+00:00 - rsyslog/run 
2021-09-18T18:53:07,817811500+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-09-18T18:53:07,838277400+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_77e992fdaf2b83b0c81ee1c90ded8d17/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-09-18T18:53:08,181306600+00:00 - iot-server/finish 1 0
2021-09-18T18:53:08,183102400+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (62)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 90
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-09-18 18:53:13,906 | root | INFO | Starting up app insights client
logging socket was 

In [26]:
# Delete the service
service.delete()